In [ ]:
# Imports

import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import os
import PIL.Image
from PIL import ImageOps
import PIL
import pathlib
import matplotlib.pyplot as plt
import datetime
import tensorboard
import IPython
import sklearn
import cv2
import subprocess
import sys

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Dataset loading

data_path = pathlib.Path('/content/drive/MyDrive/archiveX3/Training/')

data_path_test = pathlib.Path('/content/drive/MyDrive/archiveX3/Testing/')


dataset_path = tf.keras.utils.image_dataset_from_directory(
    data_path,
    labels= 'inferred',
    validation_split=0.2,
    subset='training',
    seed= 1,
    batch_size=5,
    image_size=(180, 180),
    color_mode="rgb",
    shuffle=True)

dataset_path_val = tf.keras.utils.image_dataset_from_directory(
    data_path,
    labels= 'inferred',
    validation_split=0.2,
    subset='validation',
    seed= 2,
    batch_size=5,
    image_size=(180, 180),
    color_mode="rgb",
    shuffle=True)

dataset_path_test = tf.keras.utils.image_dataset_from_directory(
    data_path_test,
    labels= 'inferred',
    seed= 3,
    batch_size=5,
    image_size=(180, 180),
    color_mode="rgb",
    shuffle=True)


In [ ]:
# catch and prefetch

AUTOTUNE=tf.data.AUTOTUNE
dataset_path = dataset_path.cache().prefetch(buffer_size=AUTOTUNE)
dataset_path_val = dataset_path_val.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
# downloading the weights

base_model = tf.keras.applications.inception_v3.InceptionV3(
    input_shape = (180, 180, 3),
    include_top = False, 
    weights = "imagenet"                   
)

In [ ]:
#saving the base_model

saving_path = pathlib.Path('/content/drive/MyDrive/archiveX3/SavedBaseModel.h5')

base_model.save(saving_path)

In [ ]:
#Model

model_path = pathlib.Path('/content/drive/MyDrive/archiveX3/SavedBaseModel.h5')

base_model = tf.keras.models.load_model(model_path)

base_model.trainable = False

rescaling = tf.keras.Sequential([
  tf.keras.layers.Rescaling(scale=1 / 127.5, offset=-1)
])
augmentation = tf.keras.Sequential([
   tf.keras.layers.RandomFlip("horizontal_and_vertical"),
   tf.keras.layers.RandomRotation(0.2)
])

inputs = tf.keras.Input(shape=(180, 180, 3))
x = augmentation(inputs)
x = rescaling(x)
x = base_model(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(256, activation='ELU', kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(512, activation='ELU', kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(512, activation='ELU', kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = tf.keras.layers.Dense(4, activation='softmax')(x)
model = tf.keras.Model(inputs, outputs)


model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0001),
    loss=tf.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy'],
    run_eagerly = True)

model.fit(
    dataset_path,
    epochs=15,   
    validation_data = dataset_path_val,
    verbose = 1)


model.evaluate(dataset_path_test, batch_size=5, verbose=1)

model.summary()

saving_path1 = pathlib.Path('/content/drive/MyDrive/archiveX3/SavedModelNoFT/')
saving_path2 = pathlib.Path('/content/drive/MyDrive/archiveX3/SavedModelWithFT/')

tf.keras.models.save_model(model,
                           saving_path1,
                           overwrite=True,
                           save_format='tf'
                           )

base_model.trainable = True

for layer in base_model.layers[:200]:
  layer.trainable = False

model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate = 0.00001),
    loss=tf.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy'],
    run_eagerly = True)

model.fit(
    dataset_path,
    epochs=25,         
    initial_epoch=15,                
    validation_data = dataset_path_val,
    verbose = 1)

model.evaluate(dataset_path_test, batch_size=5, verbose=1)

model.summary()

tf.keras.models.save_model(model,
                           saving_path2,
                           overwrite=True,
                           save_format='tf'
                           )